In [3]:
import pandas as pd
# Panda, Panda
# Panda, Panda, Panda, Panda, Panda
import numpy as np
# I got numpy in Atlanta 

## Random Forest to predict Crime 

In [103]:
INC = pd.read_csv('Data/Incidents.csv')
INC.head()

,Unnamed: 0,FID,Join_Count,TARGET_FID,Join_Cou_1,TARGET_F_1,incident_i,case_numbe,incident_d,incident_t,...,PRECIPITATIONIN,EVENTS,MEAN_VISIBILITYMILES,CityName,ZipCode2,IncidentGroup2,TotalPoverty,Weatherstr,weatherEventGroup,ZoningGroup
0,0,0,1,0,1,1,787108492,59881,12/30/2016 01:49:00 PM,INFORMATION REPORT,...,0.00,NaN,10.0,DUNWOODY,30346,Other,598.0,NaN,NaN,Office
1,1,1,1,1,1,2,787300346,59906,01/01/2017 01:00:00 AM,INFORMATION REPORT,...,NaN,NaN,NaN,DUNWOODY,30346,Other,874.0,NaN,NaN,Office
2,2,2,1,2,0,3,787147954,59892,12/31/2016 02:12:00 AM,FAMILY OFFENSE-NO VIOLENCE,...,0.17,Rain,9.0,DUNWOODY,30338,Family_Offense,66.5,Rain,Rain,
3,3,3,1,3,0,4,787174660,59898,12/31/2016 09:39:00 AM,SPEEDING,...,0.17,Rain,9.0,DUNWOODY,0,Traffic,12.0,Rain,Rain,
4,4,4,1,4,0,5,762180594,54597,06/13/2016 06:07:00 PM,FAIL TO YIELD TURNING LEFT,...,0.00,NaN,10.0,DUNWOODY,30346,Traffic,272.0,NaN,NaN,


In [104]:
INC.shape

(93014, 55)

In [105]:
INC.dtypes

Unnamed: 0                int64
FID                       int64
Join_Count                int64
TARGET_FID                int64
Join_Cou_1                int64
TARGET_F_1                int64
incident_i                int64
case_numbe                int64
incident_d               object
incident_t               object
incident_1               object
address                  object
city                     object
state                    object
zip                       int64
latitude                float64
longitude               float64
hour_of_da                int64
day_of_wee               object
parent_inc               object
date_incid               object
SOURCE_ID                 int64
Gi_Bin                    int64
B25057e1                float64
B25058e1                float64
B25059e1                float64
B25076e1                float64
B25077e1                float64
B25078e1                float64
B25018e1                float64
B25001e1                float64
B25002e1

### Select Columns for Model

In [106]:
ColstoKeep = ['incident_d', 'Join_Count', 'ZipCode2', 'latitude', 'longitude', 'hour_of_da', 'day_of_wee', 'date_incid', 
              'B25057e1', 'B25058e1', 'B25059e1', 'B25076e1', 'B25077e1', 'B25078e1', 'B25018e1', 'B25001e1', 'B25002e1', 'B25002e2', 'B25002e3', 'B19013e1',
              'EST', 'MAX_TEMPERATUREF', 'PRECIPITATIONIN', 'MEAN_VISIBILITYMILES', 
              'CityName', 'IncidentGroup2', 'TotalPoverty', 'weatherEventGroup', 'ZoningGroup']
data = INC[ColstoKeep]
data.head()

,incident_d,Join_Count,ZipCode2,latitude,longitude,hour_of_da,day_of_wee,date_incid,B25057e1,B25058e1,...,B19013e1,EST,MAX_TEMPERATUREF,PRECIPITATIONIN,MEAN_VISIBILITYMILES,CityName,IncidentGroup2,TotalPoverty,weatherEventGroup,ZoningGroup
0,12/30/2016 01:49:00 PM,1,30346,33.927104,-84.331795,13,Friday,2016-12-30 00:00:00,938.5,1083.0,...,55712.5,2016-12-30,50.0,0.00,10.0,DUNWOODY,Other,598.0,NaN,Office
1,01/01/2017 01:00:00 AM,1,30346,33.930271,-84.332347,1,Sunday,2017-01-01 00:00:00,911.0,1041.0,...,67422.0,NaN,NaN,NaN,NaN,DUNWOODY,Other,874.0,NaN,Office
2,12/31/2016 02:12:00 AM,1,30338,33.925571,-84.315793,2,Saturday,2016-12-31 00:00:00,472.0,527.0,...,85000.5,2016-12-31,47.0,0.17,9.0,DUNWOODY,Family_Offense,66.5,Rain,
3,12/31/2016 09:39:00 AM,1,0,33.943571,-84.331624,9,Saturday,2016-12-31 00:00:00,951.0,1046.5,...,111182.0,2016-12-31,47.0,0.17,9.0,DUNWOODY,Traffic,12.0,Rain,
4,06/13/2016 06:07:00 PM,1,30346,33.926443,-84.337153,18,Monday,2016-06-13 00:00:00,922.5,1108.0,...,67008.5,2016-6-13,99.0,0.00,10.0,DUNWOODY,Traffic,272.0,NaN,


Number of Incidents

In [139]:
data['Join_Count'].value_counts()
# Join Count is column from Index Grid Join

1    93012
2        2
Name: Join_Count, dtype: int64

In [140]:
INC['IncidentGroup2'].value_counts()
# Count of Crimes by type

Theft                   25150
Traffic                 17968
Other                   13026
Property_Crime          11218
Breaking_Enter           5537
Disorder                 5316
Assault                  5201
Traffic_DUI              2826
Family_Offense           2709
Drugs                    1481
Robbery                   831
Missing Person            686
Vehicle Recovery          278
Quality of Life           277
Death                     159
Weapons Offense           154
Other Sexual Offense       72
Liquor                     53
Arson                      37
Homicide                   18
Sexual Offense             16
Proactive Policing          1
Name: IncidentGroup2, dtype: int64

https://www.youtube.com/watch?v=YPItfQ87qjM

**dataM** Data Frame with Top 10 Incidents

In [110]:
# old way
# Categories[
#            (Categories.IncidentGroup2 == "Theft") | 
#            (Categories.IncidentGroup2 == "Traffic") |
#            (Categories.IncidentGroup2 == "Property Crime") |
#            (Categories.IncidentGroup2 == "Breaking & Entering") |
#            (Categories.IncidentGroup2 == "Disorder") | 
#            (Categories.IncidentGroup2 == "Assault") |
#            (Categories.IncidentGroup2 == "Traffic - DUI") |
#            (Categories.IncidentGroup2 == "Family Offense") |
#            (Categories.IncidentGroup2 == "Drugs")
# ].head(20)

# better way
valuesToKeep = ["Theft","Traffic","Property_Crime","Breaking_Enter","Disorder","Assault",
               "Traffic_DUI", "Family_Offense","Drugs", "Robbery"]

dataM = data[ data.IncidentGroup2.isin(valuesToKeep) ]
dataM.IncidentGroup2.unique()

# Keeping top 10 Crimes for model

array(['Family_Offense', 'Traffic', 'Disorder', 'Theft', 'Assault',
       'Property_Crime', 'Breaking_Enter', 'Drugs', 'Traffic_DUI',
       'Robbery'], dtype=object)

Total of 93,014 Incidents in Sandy Springs and Dunwoody from 2010 to 2016 <br/>
Select out Top 10 Crimes - left with 78,237 records

In [209]:
print data.shape
print dataM.shape

(93014, 29)
(78237, 31)


Should have spatial joined the Incident points to a Zip Code layer - I don't expect Zip Code to be significant <br/>
Zip Codes that has less than 100 Incident events were classified as 0. 

In [112]:
dataM['ZipCode2'].value_counts()

0        53665
30338     9084
30346     8772
30360     2171
30328     1979
30350     1646
30342      769
30327      151
Name: ZipCode2, dtype: int64

In [114]:
print data.weatherEventGroup.value_counts()
print dataM.weatherEventGroup.value_counts()

Rain    28900
Fog      5514
Snow      532
Name: weatherEventGroup, dtype: int64
Rain    24268
Fog      4670
Snow      448
Name: weatherEventGroup, dtype: int64


In [79]:
print "ORIGINAL {} no weather Event".format(93014 - (28900 + 5514 + 532))
print "new df {} no weather Event".format(78237 - (24268 + 4670 + 448))

ORIGINAL 58068 no weather Event
new df 48851 no weather Event


63% of Incidents occured with on a day with no "weather event" 
* Weather Event
    * Rain
    * Fog
    * Snow 

In [80]:
print "ORGINAL {}% incidents no weather event".format((58068.0/93014)*100)
print "new df {}% incidents no weather event".format((48851.0/78237)*100)

ORGINAL 62.4293117165% incidents no weather event
new df 62.439766351% incidents no weather event


In [115]:
dataM.ZoningGroup.value_counts()

                  26700
Commercial        16841
Multi_Dwelling    11899
Office             6989
Multi_Family       6421
Single_Family      4936
MIX                2295
CUP                 923
Townhouse           793
Industrial          203
Agriculture         177
NUP                  60
Name: ZoningGroup, dtype: int64

Incident points that were recorded where the X, Y fell in the road - were not spatially joined to Zoning. <br/>
Can be due to the officer writing a report from the car = no Zoning Recorded
Incident event could occur in the Road but then the report could be recorded while pulled over into a parking lot = Zoning recorded <br/><br/>
34% of Indidents have no Zoning Group 

In [22]:
print "ORIGINAL {}% incidents point landed in the road (no Zoning)".format((31866.0/93014)*100)
print "new df {}% incidents point landed in the road (no Zoning)".format((26561.0/77406)*100)

ORIGINAL 34.2593588062% incidents point landed in the road (no Zoning)
new df 34.3138774772% incidents point landed in the road (no Zoning)


In [116]:
dataM.head()

,incident_d,Join_Count,ZipCode2,latitude,longitude,hour_of_da,day_of_wee,date_incid,B25057e1,B25058e1,...,B19013e1,EST,MAX_TEMPERATUREF,PRECIPITATIONIN,MEAN_VISIBILITYMILES,CityName,IncidentGroup2,TotalPoverty,weatherEventGroup,ZoningGroup
2,12/31/2016 02:12:00 AM,1,30338,33.925571,-84.315793,2,Saturday,2016-12-31 00:00:00,472.0,527.0,...,85000.5,2016-12-31,47.0,0.17,9.0,DUNWOODY,Family_Offense,66.5,Rain,
3,12/31/2016 09:39:00 AM,1,0,33.943571,-84.331624,9,Saturday,2016-12-31 00:00:00,951.0,1046.5,...,111182.0,2016-12-31,47.0,0.17,9.0,DUNWOODY,Traffic,12.0,Rain,
4,06/13/2016 06:07:00 PM,1,30346,33.926443,-84.337153,18,Monday,2016-06-13 00:00:00,922.5,1108.0,...,67008.5,2016-6-13,99.0,0.00,10.0,DUNWOODY,Traffic,272.0,NaN,
5,06/13/2016 06:40:00 PM,1,0,33.943571,-84.331624,18,Monday,2016-06-13 00:00:00,951.0,1046.5,...,111182.0,2016-6-13,99.0,0.00,10.0,DUNWOODY,Traffic,12.0,NaN,
7,04/25/2016 01:58:00 PM,1,30346,33.925912,-84.336548,13,Monday,2016-04-25 00:00:00,922.5,1108.0,...,67008.5,2016-4-25,81.0,0.00,10.0,DUNWOODY,Disorder,272.0,NaN,Office


## Set Date to Index
### yyyy-mm-dd hh:mm:sec

In [122]:
# set "indcident_d" to index and set index as datetime.
dataM['Date'] = pd.to_datetime(dataM['incident_d'])
dataM.set_index('Date', inplace=True)

# create year column from date index
dataM['Year'] = dataM.index.year
# create month column from date index
dataM['Month'] = dataM.index.month
# create day column from date index
dataM['Day'] = dataM.index.dayofweek

C:\Users\Amber\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\Amber\Anaconda2\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Amber\Anaconda2\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/p

In [123]:
dataM.head()
# check Year, Month, Day column is there

,incident_d,Join_Count,ZipCode2,latitude,longitude,hour_of_da,day_of_wee,date_incid,B25057e1,B25058e1,...,PRECIPITATIONIN,MEAN_VISIBILITYMILES,CityName,IncidentGroup2,TotalPoverty,weatherEventGroup,ZoningGroup,Year,Month,Day
Date,,,,,,,,,,,,,,,,,,,,,
2016-12-31 02:12:00,12/31/2016 02:12:00 AM,1,30338,33.925571,-84.315793,2,Saturday,2016-12-31 00:00:00,472.0,527.0,...,0.17,9.0,DUNWOODY,Family_Offense,66.5,Rain,,2016,12,5
2016-12-31 09:39:00,12/31/2016 09:39:00 AM,1,0,33.943571,-84.331624,9,Saturday,2016-12-31 00:00:00,951.0,1046.5,...,0.17,9.0,DUNWOODY,Traffic,12.0,Rain,,2016,12,5
2016-06-13 18:07:00,06/13/2016 06:07:00 PM,1,30346,33.926443,-84.337153,18,Monday,2016-06-13 00:00:00,922.5,1108.0,...,0.00,10.0,DUNWOODY,Traffic,272.0,NaN,,2016,6,0
2016-06-13 18:40:00,06/13/2016 06:40:00 PM,1,0,33.943571,-84.331624,18,Monday,2016-06-13 00:00:00,951.0,1046.5,...,0.00,10.0,DUNWOODY,Traffic,12.0,NaN,,2016,6,0
2016-04-25 13:58:00,04/25/2016 01:58:00 PM,1,30346,33.925912,-84.336548,13,Monday,2016-04-25 00:00:00,922.5,1108.0,...,0.00,10.0,DUNWOODY,Disorder,272.0,NaN,Office,2016,4,0


In [124]:
dataM.to_csv('Data/dataM.csv')
# export dataM as dataM.csv for time series Viz

In [125]:
dfM = pd.read_csv('Data/dataM.csv')

C:\Users\Amber\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [207]:
dfM.head(1)
# Index no longer date whyyyyyy?

,Date,incident_d,Join_Count,ZipCode2,latitude,longitude,hour_of_da,day_of_wee,date_incid,B25057e1,...,PRECIPITATIONIN,MEAN_VISIBILITYMILES,CityName,IncidentGroup2,TotalPoverty,weatherEventGroup,ZoningGroup,Year,Month,Day
0,2016-12-31 02:12:00,12/31/2016 02:12:00 AM,1,30338,33.925571,-84.315793,2,Saturday,2016-12-31 00:00:00,472.0,...,0.17,9.0,DUNWOODY,Family_Offense,66.5,Rain,,2016,12,5


Now going to build Random Forest Classification Model on the Data Frame.  Reminder - these rows are Incident point from Sandy Springs and Dunwoody 

### dataM 

In [141]:
dataM.drop('incident_d', axis = 1, inplace = True)

C:\Users\Amber\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [210]:
dataM.dtypes
# ready to model
# ...maybe, hopefully ... 
# X is everything except "IncidentGroup2"
# Y is "IncidentGroup2" 10 categories 

Join_Count                int64
ZipCode2                  int64
latitude                float64
longitude               float64
hour_of_da                int64
day_of_wee               object
date_incid               object
B25057e1                float64
B25058e1                float64
B25059e1                float64
B25076e1                float64
B25077e1                float64
B25078e1                float64
B25018e1                float64
B25001e1                float64
B25002e1                float64
B25002e2                float64
B25002e3                float64
B19013e1                float64
EST                      object
MAX_TEMPERATUREF        float64
PRECIPITATIONIN          object
MEAN_VISIBILITYMILES    float64
CityName                 object
IncidentGroup2           object
TotalPoverty            float64
weatherEventGroup        object
ZoningGroup              object
Year                      int32
Month                     int32
Day                       int32
dtype: o

## dfMCats
Data Frame for Model with Dummy Variables on the Categoricial Columns including Incident Group - which are the 10 targets for the model

In [421]:
dfMCats = dataM

## Encoding Categories
Random Forest needs all the columns to contain numerical values <br/> 
All the Categories will be cast to an Integer. Each integer (aka number) corresponds to each category in a given Categorical Column <br/>
Categorical Columns still in for Model and Dummy Variables created from the Categories <br/>
The Encoded Column will serve to see if that has feature importance - the Dummy Variables from that Column identifies which values of that column was most important

In [422]:
#  ********
# lets say one column for gender has 3 values, "M", "F", "U"
# call that column gender

# genderKeys = df.gender.unique().tolist()
# genderWithInts = zip(genderKeys, range(len(genderKeys)))
# genderToIntDict = dict( genderWithInts )

# # genderToIntDict will return {"M" : 1, "F" : 2, "U" : 3}

# def genderToInt(gender):
# 	return genderToIntDict[gender]

# df["gender"] = df.gender.apply(genderToInt)
#  ******

Categorical Columns to Encode 
 * Day of the Week
 * City Name
 * Weather Event
 * Zoning Type
 * Incident Group
 * Precipitation - make sure is int

In [431]:
dfMCats.columns

Index([u'Join_Count', u'ZipCode2', u'latitude', u'longitude', u'hour_of_da',
       u'day_of_wee', u'date_incid', u'B25057e1', u'B25058e1', u'B25059e1',
       u'B25076e1', u'B25077e1', u'B25078e1', u'B25018e1', u'B25001e1',
       u'B25002e1', u'B25002e2', u'B25002e3', u'B19013e1', u'EST',
       u'MAX_TEMPERATUREF', u'PRECIPITATIONIN', u'MEAN_VISIBILITYMILES',
       u'CityName', u'IncidentGroup2', u'TotalPoverty', u'weatherEventGroup',
       u'ZoningGroup', u'Year', u'Month', u'Day', u'WeekNum', u'CityNum',
       u'WeatherEventNum', u'ZoneNum', u'IncNum'],
      dtype='object')

### Day of the Week

In [426]:
dayofWeek = dfMCats.day_of_wee.unique().tolist()
dayofWeek
# Week Starts on Friday 

['Saturday', 'Monday', 'Thursday', 'Friday', 'Tuesday', 'Sunday', 'Wednesday']

In [427]:
IndexNumforWeek = [7, 2, 5, 6, 3, 1, 4]
# Sun = 1
# Mon = 2
# Tue = 3
# Wed = 4
# Ths = 5 
# Fri = 6
# Sat = 7

In [428]:
WeekNum = zip(dayofWeek, IndexNumforWeek)
WeekNum

[('Saturday', 7),
 ('Monday', 2),
 ('Thursday', 5),
 ('Friday', 6),
 ('Tuesday', 3),
 ('Sunday', 1),
 ('Wednesday', 4)]

In [429]:
WeekNumDict = dict(WeekNum)
WeekNumDict
# brackets to curly brackets 

{'Friday': 6,
 'Monday': 2,
 'Saturday': 7,
 'Sunday': 1,
 'Thursday': 5,
 'Tuesday': 3,
 'Wednesday': 4}

In [430]:
def WeekToInt(week):
    return WeekNumDict[week]

dfMCats['WeekNum'] = dfMCats.day_of_wee.apply(WeekToInt)
dfMCats.WeekNum.unique()

C:\Users\Amber\Anaconda2\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


array([7, 2, 5, 6, 3, 1, 4], dtype=int64)

### City Name

In [432]:
dfMCats.CityName.value_counts()

SANDY SPRINGS    52195
DUNWOODY         25799
DORAVILLE          125
ATLANTA            115
GCO                  2
CHAMBLEE             1
Name: CityName, dtype: int64

In [433]:
City = dfMCats.CityName.unique().tolist()
City

['DUNWOODY', 'SANDY SPRINGS', 'ATLANTA', 'DORAVILLE', 'GCO', 'CHAMBLEE']

In [434]:
CityNum = zip(City, range(len(City)))
# CityNum
Citydict = dict(CityNum)
Citydict

{'ATLANTA': 2,
 'CHAMBLEE': 5,
 'DORAVILLE': 3,
 'DUNWOODY': 0,
 'GCO': 4,
 'SANDY SPRINGS': 1}

In [435]:
def CitytoInt(City):
    return Citydict[City]

dfMCats["CityNum"] = dfMCats.CityName.apply(CitytoInt)
dfMCats.CityNum.unique()

C:\Users\Amber\Anaconda2\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


array([0, 1, 2, 3, 4, 5], dtype=int64)

### Weather Event to Numbers

In [436]:
weatherEvent = dfMCats.weatherEventGroup.unique().tolist()
WeatherNum = dict(zip(weatherEvent, range(len(weatherEvent))))
WeatherNum

{nan: 1, 'Fog': 2, 'Rain': 0, 'Snow': 3}

In [437]:
def weathertoInt(weatherEvent):
    return WeatherNum[weatherEvent]

dfMCats['WeatherEventNum'] = dfMCats.weatherEventGroup.apply(weathertoInt)
dfMCats.WeatherEventNum.unique()

C:\Users\Amber\Anaconda2\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


array([0, 1, 2, 3], dtype=int64)

### Zoning Group

In [438]:
ZoneGroup = dfMCats.ZoningGroup.unique().tolist()
ZoneNum = dict(zip(ZoneGroup, range(len(ZoneGroup))))
ZoneNum

{' ': 0,
 'Agriculture': 6,
 'CUP': 9,
 'Commercial': 2,
 'Industrial': 10,
 'MIX': 8,
 'Multi_Dwelling': 5,
 'Multi_Family': 3,
 'NUP': 11,
 'Office': 1,
 'Single_Family': 4,
 'Townhouse': 7}

In [439]:
def ZonetoInt(Zone):
    return ZoneNum[Zone]

dfMCats['ZoneNum'] = dfMCats.ZoningGroup.apply(ZonetoInt)
dfMCats.ZoneNum.unique()

C:\Users\Amber\Anaconda2\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11], dtype=int64)

### Incident Group

In [440]:
IncGroup = dfMCats.IncidentGroup2.unique().tolist()
IncNum = dict(zip(IncGroup, range(len(IncGroup))))
IncNum

{'Assault': 4,
 'Breaking_Enter': 6,
 'Disorder': 2,
 'Drugs': 7,
 'Family_Offense': 0,
 'Property_Crime': 5,
 'Robbery': 9,
 'Theft': 3,
 'Traffic': 1,
 'Traffic_DUI': 8}

In [441]:
def InctoInt(Incident):
    return IncNum[Incident]

dfMCats['IncNum'] = dfMCats.IncidentGroup2.apply(InctoInt)
dfMCats.IncNum.unique()

C:\Users\Amber\Anaconda2\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64)

### Make Precipitation Column be Numbers
http://help.wunderground.com/knowledgebase/articles/656875-what-does-t-stand-for-on-the-rain-precipitation <br/>
Column came with float types and this frisky "T" 
```
T stands for "trace", used when precipitation has been detected, but it isn't sufficient to measure meaningfully
```

In [442]:
def castFloat(value):
    try:
        return float(value)
    except ValueError:
        return 0
    
dfMCats['PRECIPITATIONIN'] = dfMCats.PRECIPITATIONIN.apply(castFloat)
dfMCats.PRECIPITATIONIN.max()

C:\Users\Amber\Anaconda2\lib\site-packages\ipykernel\__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


4.1100000000000003

#### Alternatively Can also Enumerate all of the Columns in the DataFrame

http://stackoverflow.com/questions/24458645/label-encoding-across-multiple-columns-in-scikit-learn

In [ ]:
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
d = defaultdict(LabelEncoder)

In [203]:
# dfMCats.drop(CategoriesforEncode, axis = 1, inplace = True)
# dataM.drop('incident_d', axis = 1, inplace = True)

In [236]:
# stuff that almost works 
# CategoriesforEncode = ["day_of_wee", "CityName", "weatherEventGroup", "ZoningGroup", "IncidentGroup2"]
# dfMCatsEnc = dfMCats.join(dfMCats[CategoriesforEncode].apply(LabelEncoder().fit_transform), 
#                           how='left', rsuffix='Enc')
# dfMCatsEnc.head()

In [237]:
from collections import defaultdict
d = defaultdict(LabelEncoder)

In [248]:
Enc = dfMCats.apply(lambda x: d[x.name].fit_transform(x))
# enumerates everything in the Data Frame with fit_transform()
Enc.head()

,Join_Count,ZipCode2,latitude,longitude,hour_of_da,day_of_wee,date_incid,B25057e1,B25058e1,B25059e1,...,IncidentGroup2_Assault,IncidentGroup2_Breaking_Enter,IncidentGroup2_Disorder,IncidentGroup2_Drugs,IncidentGroup2_Family_Offense,IncidentGroup2_Property_Crime,IncidentGroup2_Robbery,IncidentGroup2_Theft,IncidentGroup2_Traffic,IncidentGroup2_Traffic_DUI
Date,,,,,,,,,,,,,,,,,,,,,
2010-01-01,0,0,13793,16550,0,0,0,128,99,153,...,0,0,0,0,0,1,0,0,0,0
2010-01-01,0,0,13793,16550,0,0,0,128,99,153,...,0,0,0,0,0,0,0,1,0,0
2010-01-01,0,0,13793,16550,0,0,0,128,99,153,...,0,0,1,0,0,0,0,0,0,0
2010-01-01,0,0,4805,8547,0,0,0,236,291,271,...,0,0,0,0,0,1,0,0,0,0
2010-01-01,0,0,4805,8547,0,0,0,236,291,271,...,0,0,0,0,0,0,0,1,0,0


In [251]:
Reverse = Enc.apply(lambda x: d[x.name].inverse_transform(x))
# And brings the original values back with inverse_transform()
Reverse.head()
# this is still cool

,Join_Count,ZipCode2,latitude,longitude,hour_of_da,day_of_wee,date_incid,B25057e1,B25058e1,B25059e1,...,IncidentGroup2_Assault,IncidentGroup2_Breaking_Enter,IncidentGroup2_Disorder,IncidentGroup2_Drugs,IncidentGroup2_Family_Offense,IncidentGroup2_Property_Crime,IncidentGroup2_Robbery,IncidentGroup2_Theft,IncidentGroup2_Traffic,IncidentGroup2_Traffic_DUI
Date,,,,,,,,,,,,,,,,,,,,,
2010-01-01,1,0,33.965092,-84.314529,0,Friday,2010-01-01 00:00:00,683.0,741.5,1000.5,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2010-01-01,1,0,33.965092,-84.314529,0,Friday,2010-01-01 00:00:00,683.0,741.5,1000.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2010-01-01,1,0,33.965092,-84.314529,0,Friday,2010-01-01 00:00:00,683.0,741.5,1000.5,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2010-01-01,1,0,33.919469,-84.359070,0,Friday,2010-01-01 00:00:00,834.0,1115.5,1315.5,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2010-01-01,1,0,33.919469,-84.359070,0,Friday,2010-01-01 00:00:00,834.0,1115.5,1315.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [443]:
print dfMCats.shape
print dfMCatEnc.shape
print Enc.shape

(78237, 36)
(85761, 69)
(85761, 69)


### Make Dummy Variables on the Categorical Columns

In [444]:
# Making Dummies after Encoding so that the Incident Group dummy variables go at the end of the Data Frame
CategoriesforDummies = ["day_of_wee", "CityName", "weatherEventGroup", "ZoningGroup", "IncidentGroup2"]
Categories = dfMCats[CategoriesforDummies]
print Categories.head()
dummies = pd.get_dummies(Categories)
# dummies.head()
dfMCats = dfMCats.join(dummies)
dfMCats.head()

                    day_of_wee  CityName weatherEventGroup ZoningGroup  \
Date                                                                     
2016-12-31 02:12:00   Saturday  DUNWOODY              Rain               
2016-12-31 09:39:00   Saturday  DUNWOODY              Rain               
2016-06-13 18:07:00     Monday  DUNWOODY               NaN               
2016-06-13 18:40:00     Monday  DUNWOODY               NaN               
2016-04-25 13:58:00     Monday  DUNWOODY               NaN      Office   

                     IncidentGroup2  
Date                                 
2016-12-31 02:12:00  Family_Offense  
2016-12-31 09:39:00         Traffic  
2016-06-13 18:07:00         Traffic  
2016-06-13 18:40:00         Traffic  
2016-04-25 13:58:00        Disorder  


,Join_Count,ZipCode2,latitude,longitude,hour_of_da,day_of_wee,date_incid,B25057e1,B25058e1,B25059e1,...,IncidentGroup2_Assault,IncidentGroup2_Breaking_Enter,IncidentGroup2_Disorder,IncidentGroup2_Drugs,IncidentGroup2_Family_Offense,IncidentGroup2_Property_Crime,IncidentGroup2_Robbery,IncidentGroup2_Theft,IncidentGroup2_Traffic,IncidentGroup2_Traffic_DUI
Date,,,,,,,,,,,,,,,,,,,,,
2010-01-01,1,0,33.965092,-84.314529,0,Friday,2010-01-01 00:00:00,683.0,741.5,1000.5,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2010-01-01,1,0,33.965092,-84.314529,0,Friday,2010-01-01 00:00:00,683.0,741.5,1000.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2010-01-01,1,0,33.965092,-84.314529,0,Friday,2010-01-01 00:00:00,683.0,741.5,1000.5,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2010-01-01,1,0,33.919469,-84.359070,0,Friday,2010-01-01 00:00:00,834.0,1115.5,1315.5,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2010-01-01,1,0,33.919469,-84.359070,0,Friday,2010-01-01 00:00:00,834.0,1115.5,1315.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


### Drop a few things 
Drop Categorical Columns 

In [445]:
print dataM.shape
print dfMCats.shape

(78237, 36)
(85761, 74)


In [446]:
CatColstoDrop = ["day_of_wee", "CityName", "weatherEventGroup", "ZoningGroup", "EST", "date_incid", "IncidentGroup2"]
# drop these columns that have text categories for the model
dfMCats.drop(CatColstoDrop, axis = 1, inplace = True)

In [447]:
dfMCats.columns

Index([u'Join_Count', u'ZipCode2', u'latitude', u'longitude', u'hour_of_da',
       u'B25057e1', u'B25058e1', u'B25059e1', u'B25076e1', u'B25077e1',
       u'B25078e1', u'B25018e1', u'B25001e1', u'B25002e1', u'B25002e2',
       u'B25002e3', u'B19013e1', u'MAX_TEMPERATUREF', u'PRECIPITATIONIN',
       u'MEAN_VISIBILITYMILES', u'TotalPoverty', u'Year', u'Month', u'Day',
       u'WeekNum', u'CityNum', u'WeatherEventNum', u'ZoneNum', u'IncNum',
       u'day_of_wee_Friday', u'day_of_wee_Monday', u'day_of_wee_Saturday',
       u'day_of_wee_Sunday', u'day_of_wee_Thursday', u'day_of_wee_Tuesday',
       u'day_of_wee_Wednesday', u'CityName_ATLANTA', u'CityName_CHAMBLEE',
       u'CityName_DORAVILLE', u'CityName_DUNWOODY', u'CityName_GCO',
       u'CityName_SANDY SPRINGS', u'weatherEventGroup_Fog',
       u'weatherEventGroup_Rain', u'weatherEventGroup_Snow', u'ZoningGroup_ ',
       u'ZoningGroup_Agriculture', u'ZoningGroup_CUP',
       u'ZoningGroup_Commercial', u'ZoningGroup_Industrial',
     

### Get rid on NaN
- Random Forest Don't Like

In [465]:
dfMCats.fillna(0, inplace = True)

## Test - Train Split the Data

In [466]:
cols = dfMCats.columns
for i, cols in enumerate(cols):
    print(i, cols)

(0, 'Join_Count')
(1, 'ZipCode2')
(2, 'latitude')
(3, 'longitude')
(4, 'hour_of_da')
(5, 'B25057e1')
(6, 'B25058e1')
(7, 'B25059e1')
(8, 'B25076e1')
(9, 'B25077e1')
(10, 'B25078e1')
(11, 'B25018e1')
(12, 'B25001e1')
(13, 'B25002e1')
(14, 'B25002e2')
(15, 'B25002e3')
(16, 'B19013e1')
(17, 'MAX_TEMPERATUREF')
(18, 'PRECIPITATIONIN')
(19, 'MEAN_VISIBILITYMILES')
(20, 'TotalPoverty')
(21, 'Year')
(22, 'Month')
(23, 'Day')
(24, 'WeekNum')
(25, 'CityNum')
(26, 'WeatherEventNum')
(27, 'ZoneNum')
(28, 'IncNum')
(29, 'day_of_wee_Friday')
(30, 'day_of_wee_Monday')
(31, 'day_of_wee_Saturday')
(32, 'day_of_wee_Sunday')
(33, 'day_of_wee_Thursday')
(34, 'day_of_wee_Tuesday')
(35, 'day_of_wee_Wednesday')
(36, 'CityName_ATLANTA')
(37, 'CityName_CHAMBLEE')
(38, 'CityName_DORAVILLE')
(39, 'CityName_DUNWOODY')
(40, 'CityName_GCO')
(41, 'CityName_SANDY SPRINGS')
(42, 'weatherEventGroup_Fog')
(43, 'weatherEventGroup_Rain')
(44, 'weatherEventGroup_Snow')
(45, 'ZoningGroup_ ')
(46, 'ZoningGroup_Agriculture')

In [353]:
dfMCats.IncNum.dtypes

dtype('int64')

In [467]:
from sklearn.cross_validation import train_test_split
X = dfMCats[dfMCats.columns[1:57]]
# hmm yes this is a problem "IncNum" Column is a target and predictor....
y = dfMCats.IncNum
yAssault = dfMCats.IncidentGroup2_Assault
yBreak_Enter = dfMCats.IncidentGroup2_Breaking_Enter
yDisorder = dfMCats.IncidentGroup2_Disorder
yDrugs = dfMCats.IncidentGroup2_Drugs
yFamilyOff = dfMCats.IncidentGroup2_Family_Offense
yPropCrime = dfMCats.IncidentGroup2_Property_Crime
yRobbery = dfMCats.IncidentGroup2_Robbery
yTheft = dfMCats.IncidentGroup2_Theft
yTraffic = dfMCats.IncidentGroup2_Traffic
yDUI = dfMCats.IncidentGroup2_Traffic_DUI
# ****
# there has to be a better way!!!
# ****
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)
X_train, X_test, y_train_Assault, y_test_Assault = train_test_split(X, yAssault, test_size=0.30, random_state=42)
X_train, X_test, y_train_BreakEnter, y_test_BreakEnter = train_test_split(X, yBreak_Enter, test_size=0.30, random_state=42)
X_train, X_test, y_train_Disorder, y_test_Disorder = train_test_split(X, yDisorder, test_size=0.30, random_state=42)
X_train, X_test, y_train_Drugs, y_test_Drugs = train_test_split(X, yDrugs, test_size=0.30, random_state=42)
X_train, X_test, y_train_FamOffense, y_test_FamOffense = train_test_split(X, yFamilyOff, test_size=0.30, random_state=42)
X_train, X_test, y_train_PropCrime, y_test_PropCrime = train_test_split(X, yPropCrime, test_size=0.30, random_state=42)
X_train, X_test, y_train_Robbery, y_test_Robbery = train_test_split(X, yRobbery, test_size=0.30, random_state=42)
X_train, X_test, y_train_Theft, y_test_Theft = train_test_split(X, yTheft, test_size=0.30, random_state=42)
X_train, X_test, y_train_Traffic, y_test_Traffic = train_test_split(X, yTraffic, test_size=0.30, random_state=42)
X_train, X_test, y_train_DUI, y_test_DUI = train_test_split(X, yDUI, test_size=0.30, random_state=42)

In [460]:
X.columns

Index([u'ZipCode2', u'latitude', u'longitude', u'hour_of_da', u'B25057e1',
       u'B25058e1', u'B25059e1', u'B25076e1', u'B25077e1', u'B25078e1',
       u'B25018e1', u'B25001e1', u'B25002e1', u'B25002e2', u'B25002e3',
       u'B19013e1', u'MAX_TEMPERATUREF', u'PRECIPITATIONIN',
       u'MEAN_VISIBILITYMILES', u'TotalPoverty', u'Year', u'Month', u'Day',
       u'WeekNum', u'CityNum', u'WeatherEventNum', u'ZoneNum', u'IncNum',
       u'day_of_wee_Friday', u'day_of_wee_Monday', u'day_of_wee_Saturday',
       u'day_of_wee_Sunday', u'day_of_wee_Thursday', u'day_of_wee_Tuesday',
       u'day_of_wee_Wednesday', u'CityName_ATLANTA', u'CityName_CHAMBLEE',
       u'CityName_DORAVILLE', u'CityName_DUNWOODY', u'CityName_GCO',
       u'CityName_SANDY SPRINGS', u'weatherEventGroup_Fog',
       u'weatherEventGroup_Rain', u'weatherEventGroup_Snow', u'ZoningGroup_ ',
       u'ZoningGroup_Agriculture', u'ZoningGroup_CUP',
       u'ZoningGroup_Commercial', u'ZoningGroup_Industrial',
       u'ZoningGroup

### Here it goes - finally, the fun part - fitting data to Random Forest Classification
http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html<br/>
http://scikit-learn.org/stable/modules/multiclass.html<br/>
http://scikit-learn.org/dev/modules/generated/sklearn.preprocessing.LabelEncoder.html <br/>
http://scikit-learn.org/dev/modules/generated/sklearn.preprocessing.OneHotEncoder.html#sklearn.preprocessing.OneHotEncoder <br/>
http://stackoverflow.com/questions/24458645/label-encoding-across-multiple-columns-in-scikit-learn

In [461]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# Magic with the DecisionTree, do voodoo (magic)
# I'm cookin, I'm cookin, in the kitchen with pots and pans 

In [462]:
dfMCats.head(10)

,Join_Count,ZipCode2,latitude,longitude,hour_of_da,B25057e1,B25058e1,B25059e1,B25076e1,B25077e1,...,IncidentGroup2_Assault,IncidentGroup2_Breaking_Enter,IncidentGroup2_Disorder,IncidentGroup2_Drugs,IncidentGroup2_Family_Offense,IncidentGroup2_Property_Crime,IncidentGroup2_Robbery,IncidentGroup2_Theft,IncidentGroup2_Traffic,IncidentGroup2_Traffic_DUI
Date,,,,,,,,,,,,,,,,,,,,,
2010-01-01 00:00:00,1,0,33.965092,-84.314529,0,683.0,741.5,1000.5,403250.0,507400.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2010-01-01 00:00:00,1,0,33.965092,-84.314529,0,683.0,741.5,1000.5,403250.0,507400.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2010-01-01 00:00:00,1,0,33.965092,-84.314529,0,683.0,741.5,1000.5,403250.0,507400.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2010-01-01 00:00:00,1,0,33.919469,-84.359070,0,834.0,1115.5,1315.5,159500.0,226500.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2010-01-01 00:00:00,1,0,33.919469,-84.359070,0,834.0,1115.5,1315.5,159500.0,226500.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2010-01-01 00:00:00,1,0,33.919469,-84.359070,0,834.0,1115.5,1315.5,159500.0,226500.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2010-01-01 00:00:00,1,0,33.916951,-84.435593,0,0.0,0.0,0.0,546700.0,710600.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2010-01-01 00:00:00,1,0,33.916951,-84.435593,0,0.0,0.0,0.0,546700.0,710600.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2010-01-01 00:00:00,1,0,33.916951,-84.435593,0,0.0,0.0,0.0,546700.0,710600.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


* X = dfMCats[dfMCats.columns[1:59]]

* y = dfMCats.IncNum
* yAssault = dfMCats.IncidentGroup2_Assault
* yBreak_Enter = dfMCats.IncidentGroup2_Breaking_Enter
* yDisorder = dfMCats.IncidentGroup2_Disorder
* yDrugs = dfMCats.IncidentGroup2_Drugs
* yFamilyOff = dfMCats.IncidentGroup2_Family_Offense
* yPropCrime = dfMCats.IncidentGroup2_Property_Crime
* yRobbery = dfMCats.IncidentGroup2_Robbery
* yTheft = dfMCats.IncidentGroup2_Theft
* yTraffic = dfMCats.IncidentGroup2_Traffic
* yDUI = dfMCats.IncidentGroup2_Traffic_DUI

In [470]:
# Fit Decision Tree
modelDT = DecisionTreeClassifier()
modelDT_Assault = DecisionTreeClassifier()
modelDT_BreakEnter = DecisionTreeClassifier()
modelDT_Disorder =DecisionTreeClassifier()
modelDT_Drugs = DecisionTreeClassifier()
modelDT_FamOff = DecisionTreeClassifier()
modelDT_PropCrime = DecisionTreeClassifier()
modelDT_Robbery = DecisionTreeClassifier()
modelDT_Theft = DecisionTreeClassifier()
modelDT_Traffic = DecisionTreeClassifier()
modelDT_DUI = DecisionTreeClassifier()

modelDT.fit(X, y)
modelDT_Assault.fit(X, yAssault)
modelDT_BreakEnter.fit(X, yBreak_Enter) 
modelDT_Disorder.fit(X, yDisorder)
modelDT_Drugs.fit(X, yDrugs)
modelDT_FamOff.fit(X, yFamilyOff)
modelDT_PropCrime.fit(X, yPropCrime)
modelDT_Robbery.fit(X, yRobbery)
modelDT_Theft.fit(X, yTheft)
modelDT_Traffic.fit(X, yTraffic)
modelDT_DUI.fit(X, yDUI)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

In [239]:
dfMCats.columns

Index([u'Join_Count', u'ZipCode2', u'latitude', u'longitude', u'hour_of_da',
       u'day_of_wee', u'date_incid', u'B25057e1', u'B25058e1', u'B25059e1',
       u'B25076e1', u'B25077e1', u'B25078e1', u'B25018e1', u'B25001e1',
       u'B25002e1', u'B25002e2', u'B25002e3', u'B19013e1', u'EST',
       u'MAX_TEMPERATUREF', u'PRECIPITATIONIN', u'MEAN_VISIBILITYMILES',
       u'CityName', u'IncidentGroup2', u'TotalPoverty', u'weatherEventGroup',
       u'ZoningGroup', u'Year', u'Month', u'Day', u'day_of_wee_Friday',
       u'day_of_wee_Monday', u'day_of_wee_Saturday', u'day_of_wee_Sunday',
       u'day_of_wee_Thursday', u'day_of_wee_Tuesday', u'day_of_wee_Wednesday',
       u'CityName_ATLANTA', u'CityName_CHAMBLEE', u'CityName_DORAVILLE',
       u'CityName_DUNWOODY', u'CityName_GCO', u'CityName_SANDY SPRINGS',
       u'weatherEventGroup_Fog', u'weatherEventGroup_Rain',
       u'weatherEventGroup_Snow', u'ZoningGroup_ ', u'ZoningGroup_Agriculture',
       u'ZoningGroup_CUP', u'ZoningGroup_Comm

In [496]:
Preds = X.columns.tolist()

In [495]:
FI = modelDT.feature_importances_
FI_Assault = modelDT_Assault.feature_importances_
FI_BreakEnter = modelDT_BreakEnter.feature_importances_ 
FI_Disorder = modelDT_Disorder.feature_importances_
FI_Drugs = modelDT_Drugs.feature_importances_
FI_FamOff = modelDT_FamOff.feature_importances_
FI_PropCrime = modelDT_PropCrime.feature_importances_
FI_Robbery = modelDT_Robbery.feature_importances_
FI_Theft = modelDT_Theft.feature_importances_
FI_Traffic = modelDT_Traffic.feature_importances_
FI_DUI = modelDT_DUI.feature_importances_


dic = {"Predictors": pd.Series(Preds), "Assault": pd.Series(FI_Assault), "BreakEnter" : pd.Series(FI_BreakEnter),
      "Disorder": pd.Series(FI_Disorder), "Drugs": pd.Series(FI_Drugs), "FamilyOffense" : pd.Series(FI_FamOff),
      "PropertyCrime": pd.Series(FI_PropCrime), "Robbery" : pd.Series(FI_Robbery), "Theft" : pd.Series(FI_Theft),
      "Traffic": pd.Series(FI_Traffic), "DUI": pd.Series(FI_DUI)
      }
FIdf = pd.DataFrame(dic)
FIdf.set_index('Predictors', inplace = True)
FIdf

,Assault,BreakEnter,DUI,Disorder,Drugs,FamilyOffense,PropertyCrime,Robbery,Theft,Traffic
Predictors,,,,,,,,,,
ZipCode2,0.000673,0.000572,0.000000,3.207301e-04,0.000806,0.000740,0.001282,0.000000e+00,0.000792,0.000551
latitude,0.004458,0.005723,0.000176,2.303092e-03,0.001630,0.002942,0.004628,4.263503e-03,0.005052,0.002351
longitude,0.003471,0.007887,0.001420,2.549412e-03,0.002480,0.004366,0.005496,1.703002e-03,0.005394,0.002158
hour_of_da,0.003230,0.006397,0.000143,1.584328e-03,0.000210,0.004240,0.005849,9.701936e-04,0.006304,0.002351
B25057e1,0.000554,0.001995,0.000334,1.631880e-03,0.001718,0.000470,0.001738,5.117755e-03,0.000922,0.000488
B25058e1,0.000757,0.000904,0.000126,6.669698e-04,0.000337,0.000454,0.001624,1.962314e-03,0.000909,0.000526
B25059e1,0.001285,0.001203,0.000000,5.287816e-04,0.000163,0.000848,0.000912,8.101736e-04,0.001564,0.000859
B25076e1,0.001240,0.000926,0.000000,6.341534e-04,0.000989,0.001622,0.001886,0.000000e+00,0.001562,0.000635
B25077e1,0.000035,0.000431,0.000407,8.144651e-04,0.000223,0.000641,0.001435,0.000000e+00,0.001416,0.000624


```
----------
estimators_ : list of DecisionTreeClassifier
    The collection of fitted sub-estimators.

classes_ : array of shape = [n_classes] or a list of such arrays
    The classes labels (single output problem), or a list of arrays of
    class labels (multi-output problem).

n_classes_ : int or list
    The number of classes (single output problem), or a list containing the
    number of classes for each output (multi-output problem).

n_features_ : int
    The number of features when ``fit`` is performed.

n_outputs_ : int
    The number of outputs when ``fit`` is performed.

feature_importances_ : array of shape = [n_features]
    The feature importances (the higher, the more important the feature).

oob_score_ : float
    Score of the training dataset obtained using an out-of-bag estimate.

oob_decision_function_ : array of shape = [n_samples, n_classes]
    Decision function computed with out-of-bag estimate on the training
    set. If n_estimators is small it might be possible that a data point
    was never left out during the bootstrap. In this case,
    `oob_decision_function_` might contain NaN.

References
----------

.. [1] L. Breiman, "Random Forests", Machine Learning, 45(1), 5-32, 2001.
```

In [497]:
modelRF = RandomForestClassifier()
modelRF_Assault = RandomForestClassifier()
modelRF_BreakEnter = RandomForestClassifier()
modelRF_Disorder = RandomForestClassifier()
modelRF_Drugs = RandomForestClassifier()
modelRF_FamOff = RandomForestClassifier()
modelRF_PropCrime = RandomForestClassifier()
modelRF_Robbery = RandomForestClassifier()
modelRF_Theft = RandomForestClassifier()
modelRF_Traffic = RandomForestClassifier()
modelRF_DUI = RandomForestClassifier()

modelRF.fit(X, y)
modelRF_Assault.fit(X, yAssault)
modelRF_BreakEnter.fit(X, yBreak_Enter) 
modelRF_Disorder.fit(X, yDisorder)
modelRF_Drugs.fit(X, yDrugs)
modelRF_FamOff.fit(X, yFamilyOff)
modelRF_PropCrime.fit(X, yPropCrime)
modelRF_Robbery.fit(X, yRobbery)
modelRF_Theft.fit(X, yTheft)
modelRF_Traffic.fit(X, yTraffic)
modelRF_DUI.fit(X, yDUI)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [498]:
FIRF = modelDT.feature_importances_
FIRF_Assault = modelRF_Assault.feature_importances_
FIRF_BreakEnter = modelRF_BreakEnter.feature_importances_ 
FIRF_Disorder = modelRF_Disorder.feature_importances_
FIRF_Drugs = modelRF_Drugs.feature_importances_
FIRF_FamOff = modelRF_FamOff.feature_importances_
FIRF_PropCrime = modelRF_PropCrime.feature_importances_
FIRF_Robbery = modelRF_Robbery.feature_importances_
FIRF_Theft = modelRF_Theft.feature_importances_
FIRF_Traffic = modelRF_Traffic.feature_importances_
FIRF_DUI = modelRF_DUI.feature_importances_

dic = {"Predictors": pd.Series(Preds), "Assault": pd.Series(FIRF_Assault), "BreakEnter" : pd.Series(FIRF_BreakEnter),
      "Disorder": pd.Series(FIRF_Disorder), "Drugs": pd.Series(FIRF_Drugs), "FamilyOffense" : pd.Series(FIRF_FamOff),
      "PropertyCrime": pd.Series(FIRF_PropCrime), "Robbery" : pd.Series(FIRF_Robbery), "Theft" : pd.Series(FIRF_Theft),
      "Traffic": pd.Series(FIRF_Traffic), "DUI": pd.Series(FIRF_DUI)
      }
FIRFdf = pd.DataFrame(dic)
FIRFdf.set_index('Predictors', inplace = True)
FIRFdf

,Assault,BreakEnter,DUI,Disorder,Drugs,FamilyOffense,PropertyCrime,Robbery,Theft,Traffic
Predictors,,,,,,,,,,
ZipCode2,0.004764,0.004234,0.003273,0.003705,0.004975,0.002018,0.005003,0.001929,0.008305,0.004355
latitude,0.030569,0.028500,0.018487,0.026840,0.021964,0.016429,0.025223,0.015405,0.023269,0.018890
longitude,0.030904,0.024404,0.015927,0.025554,0.026791,0.017492,0.024842,0.014248,0.026493,0.015446
hour_of_da,0.035622,0.047419,0.077894,0.027653,0.043363,0.016175,0.037389,0.014292,0.029865,0.016910
B25057e1,0.005794,0.006655,0.004441,0.005146,0.006093,0.002954,0.005050,0.003819,0.014722,0.005069
B25058e1,0.007658,0.006488,0.005744,0.005630,0.006427,0.002819,0.006835,0.003442,0.012040,0.003311
B25059e1,0.006126,0.006601,0.006082,0.006293,0.005046,0.003562,0.006208,0.002693,0.006424,0.004446
B25076e1,0.006877,0.006666,0.006219,0.006546,0.007487,0.003216,0.006059,0.003306,0.006111,0.004757
B25077e1,0.006312,0.006220,0.007917,0.007901,0.006666,0.002840,0.006765,0.002635,0.009902,0.004474
